# Basic Tasks using spannerlib


In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
# importing dependencies
import re
import pandas as pd
from pandas import DataFrame
from pathlib import Path
from spannerlib import get_magic_session,Session,Span


This tutorials aim to show how to use the spannerlib framework for simple use cases.
To illustrate the simplicity of using spannerlib, all required IE functions will be implemented from scratch.

## Finding identical sentences in a corpus of documents

Imagine we have a collection of documents, which have a document id (perhaps identified by a path on a file system),
and a content. And imaging we would like to find identical sentences in different in these documents.

Finding identical sentences is a toy example that can be extended to finding identical entity mentions etc...

In [ ]:
input_documents = pd.DataFrame([
    ('doc1', 'The quick brown fox jumps over the lazy dog. Im walking on Sunshine.'),
    ('doc2', 'Im walking on Sunshine. Lorem ipsum. Im walking on Sunshine.'),
    ('doc3', 'All you need is love. The quick brown fox jumps over the lazy dog.'),
])
input_documents

,0,1
0,doc1,The quick brown fox jumps over the lazy dog. I...
1,doc2,Im walking on Sunshine. Lorem ipsum. Im walkin...
2,doc3,All you need is love. The quick brown fox jump...


To do so, we would need 2 IE functions,
* Ones that extract the Span of a sentence from a document, called `split`
* The other which would let us know when sentences have identical content but are not actually the same sentence.
  * this will require them to have the same content when ignoring whitespace
  * but not be equal spans. Lets call this function `eq_content_spans`

Lets implement them and register them to our session object. We will use the `Span` class to save indexed substrings.



In [ ]:
' Im walking on Sunshine. '.strip()

'Im walking on Sunshine.'

In [ ]:
# this implementation is naive
# the standard library has a rgx_split ie function that does this in a more efficient way
def split(text):
    split_indices = [ pos for pos,char in enumerate(text) if char == '.' ]
    start = 0
    for pos,char in enumerate(text):
        if char == '.':
            yield Span(text, start, pos)
            start = pos+1

print(list(split('The quick brown fox jumps over the lazy dog. Im walking on Sunshine.')))

def eq_content_spans(span1, span2):
    # notice that we are yielding a boolean value
    yield span1 != span2 and str(span1).strip() == str(span2).strip()

# we register the functions and their input and output schema
sess = get_magic_session()
sess.register('split', split, [str],[Span])
sess.register('eq_content_spans', eq_content_spans,[Span,Span],[bool])


[[@3ba775,0,43) "The quick ...", [@3ba775,44,67) " Im walkin..."]


Now let us import our data

In [ ]:
sess.import_rel('Docs', input_documents)

Lets make sure we can see our data in using Spannerlog

In [ ]:
%%spannerlog
?Docs(doc_id,text)

'?Docs(doc_id,text)'

,doc_id,text
0,doc1,The quick brown fox jumps over the lazy dog. Im walking on Sunshine.
1,doc2,Im walking on Sunshine. Lorem ipsum. Im walking on Sunshine.
2,doc3,All you need is love. The quick brown fox jumps over the lazy dog.


Now let us build rules that allow us to find identical sentences:

In [ ]:
%%spannerlog
# this rule gives us
Sents(doc_id,sent)<-\
    Docs(doc_id,text),split(text)->(sent)
?Sents(doc_id,sent)

# this rule find equal pairs of sentences
EqualSents(doc_id1,sent1,doc_id2,sent2)<-\
    Sents(doc_id1,sent1),\
    Sents(doc_id2,sent2),\
    eq_content_spans(sent1,sent2)->(True)
?EqualSents(doc_id1,sent1,doc_id2,sent2)

'?Sents(doc_id,sent)'

,doc_id,sent
0,doc1,"[@3ba775,0,43) ""The quick ..."""
1,doc1,"[@3ba775,44,67) "" Im walkin..."""
2,doc2,"[@06bc2d,0,22) ""Im walking..."""
3,doc2,"[@06bc2d,23,35) "" Lorem ips..."""
4,doc2,"[@06bc2d,36,59) "" Im walkin..."""
5,doc3,"[@9c32df,0,20) ""All you ne..."""
6,doc3,"[@9c32df,21,65) "" The quick..."""


'?EqualSents(doc_id1,sent1,doc_id2,sent2)'

,doc_id1,sent1,doc_id2,sent2
0,doc1,"[@3ba775,0,43) ""The quick ...""",doc3,"[@9c32df,21,65) "" The quick..."""
1,doc1,"[@3ba775,44,67) "" Im walkin...""",doc2,"[@06bc2d,0,22) ""Im walking..."""
2,doc1,"[@3ba775,44,67) "" Im walkin...""",doc2,"[@06bc2d,36,59) "" Im walkin..."""
3,doc2,"[@06bc2d,0,22) ""Im walking...""",doc1,"[@3ba775,44,67) "" Im walkin..."""
4,doc2,"[@06bc2d,0,22) ""Im walking...""",doc2,"[@06bc2d,36,59) "" Im walkin..."""
5,doc2,"[@06bc2d,36,59) "" Im walkin...""",doc1,"[@3ba775,44,67) "" Im walkin..."""
6,doc2,"[@06bc2d,36,59) "" Im walkin...""",doc2,"[@06bc2d,0,22) ""Im walking..."""
7,doc3,"[@9c32df,21,65) "" The quick...""",doc1,"[@3ba775,0,43) ""The quick ..."""


Notice that we got each pair twice. This is because while we think of a sentence pair as a set with two sentences,
the tuples `(x,y)` and `(y,x)` are actually different. To remedy this we can limit our pairs to pairs where the first sentence is in a smaller position (ie position in memory).

We can do that by introducing a `span_lt` ie function

In [ ]:
def span_lt(span1, span2):
    yield span1 < span2

sess.register('span_lt', span_lt, [Span,Span],[bool])

In [ ]:
%%spannerlog
EqualSentsUniqe(doc_id1,sent1,doc_id2,sent2)<-\
    Sents(doc_id1,sent1),\
    Sents(doc_id2,sent2),\
    span_lt(sent1,sent2)->(True),\
    eq_content_spans(sent1,sent2)->(True)
    
?EqualSentsUniqe(doc_id1,sent1,doc_id2,sent2)

'?EqualSentsUniqe(doc_id1,sent1,doc_id2,sent2)'

,doc_id1,sent1,doc_id2,sent2
0,doc2,"[@06bc2d,0,22) ""Im walking...""",doc1,"[@3ba775,44,67) "" Im walkin..."""
1,doc2,"[@06bc2d,0,22) ""Im walking...""",doc2,"[@06bc2d,36,59) "" Im walkin..."""
2,doc2,"[@06bc2d,36,59) "" Im walkin...""",doc1,"[@3ba775,44,67) "" Im walkin..."""
3,doc3,"[@9c32df,21,65) "" The quick...""",doc1,"[@3ba775,0,43) ""The quick ..."""


In [ ]:
# # cleaning the session before the next use case
# sess.clear()

### Calling an LLMs as part of a data dependant pipeline

In this use case, we will implement basic llm based pipelines using spannerlib.

First of all, we need to wrap some llm api as an IE function.
We want an ie function that takes a string and returns a string.
Since most LLM api expect a dict of the form `{'role':role,'content':message}`
we will right converters that parse them from prompt strings of the form 
```txt
role: content
role: content
```

In [ ]:
# load openAI api key for env file

from dotenv import load_dotenv
load_dotenv('.env_dev')
import os
assert os.getenv('OPENAI_API_KEY') is not None

In [ ]:
from spannerlib.ie_func.basic import rgx_split
from functools import cache
import openai
from joblib import Memory
memory = Memory("cachedir", verbose=0)
client = openai.Client()

# we use the rgx_split function to split the string into messages
def str_to_messages (string_prompt):
    return [
        {
            'role': str(role).replace(': ',''),
            'content': str(content)
        } for role,content in rgx_split('system:\s|assistant:\s|user:\s', string_prompt.strip())
    ]
def messages_to_string(msgs):
    return ''.join([f"{msg['content']}" for msg in msgs])

# the specific API we are going to call using the messages interface
def openai_chat(model, messages):
    respone = client.chat.completions.create(
        model=model,
        messages=messages,
        seed=42
    )
    return [dict(respone.choices[0].message)]

# we disk cache our function to spare my openAI credits
@memory.cache
def llm(model, question):
    q_msgs = str_to_messages(question)
    a_msgs = openai_chat(model, q_msgs)
    answer = messages_to_string(a_msgs)
    return answer

def llm_ie(model, question):
    return [llm(model, question)]




In [ ]:
llm('gpt-3.5-turbo','user: Hello, who are you?')

'Hello! I am a AI-powered virtual assistant designed to help and provide information to users. How can I assist you today?'

Now we can register the llm function as an ie function and use it from in spannerlog code.

In [ ]:
sess.register('llm', llm_ie, [str,str],[str])

In [ ]:
%%spannerlog

model = 'gpt-3.5-turbo'
prompt = "user: sing it with me, love, what is it good for?"

TestLLM(answer)<-\
    llm($model,$prompt)->(answer)

?TestLLM(answer)


'?TestLLM(answer)'

,answer
0,Absolutely nothing!


Now let see how we can use spannerlib to build a simple pipeline that varries the LLM prompt in a data dependant way.
Imagine that we have a chatbot that we want to act differently based on the topic of conversation and the user preference.

To enable us to format prompts from data, we will use a prompt formatting function with a `printf` like syntax.
It is available in spannerlib's stdlib, but we will reproduce it here.

In [ ]:
def format_ie(f_string,*params):
    yield f_string.format(*params),

# note that since the schema is dynamic we need to define a function that returns the schema based on the arity
string_schema = lambda x: ([str]*x)

sess.register('format', format_ie, string_schema,[str])



We will model our data as follows:

In [ ]:

# a binary relation that stores how the user prefers the formatted
user_answer_prefernce = pd.DataFrame([
    ('Bob', 'please answer in prose. Make sure you add references in the end like a bibliography.'),
    ('Joe', 'Prose is hard for me to read quickly. Format the answer in bullet points.'),
    ('Sally', 'Try to avoid complicated jargon and use simple language.'),
])

# relation that stores per topic style
topic_specific_style = pd.DataFrame([
    ('history', 'please answer in a narrative form, use shakespearing language and lots of examples'),
    ('science', 'Use mathematical notation and formulas to explain the concepts.'),
    ('hiphop', 'introduce the answer with a rap verse.'),
    ('other', 'Be polite and neutral in your answer. Avoid controversial topics.'),
])

sess.import_rel('UserAnswerPreference', user_answer_prefernce)
sess.import_rel('TopicSpecificStyle', topic_specific_style)

Now we will build our pipeline as follows:
* We will make an llm call to help us decide the topic of the question
* Based on the topic and the user, we will formulate the final prompt for the llm to answer.


In [ ]:
prompts = pd.DataFrame([
    ('topic_selection',
"""
system: Please select a topic from the following list: [history, science, hiphop, other]
based on the question provided by the user. You are only allwed to say the topic name, nothing else.

user: {}
"""),
    ('custom_style_prompt',
"""
system: Answer the question of the user in the following style:

topic specific style instructions: {}

user specific style instructions: {}

user: {}
"""
)
])
sess.import_rel('Prompts', prompts)
prompts

,0,1
0,topic_selection,system: Please select a topic from the follow...
1,custom_style_prompt,system: Answer the question of the user in th...


In [ ]:
%%spannerlog
?Prompts(prompt_id,prompt)

'?Prompts(prompt_id,prompt)'

,prompt_id,prompt
0,custom_style_prompt,system: Answer the question of the user in the following style: topic specific style instructions: {} user specific style instructions: {} user: {}
1,topic_selection,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: {}"


Now given a relation of user and question

In [ ]:
questions= pd.DataFrame([
    ('Bob', 'Who won the civil war?'),
    ('Joe', 'Who won the civil war?'),
    ('Sally', 'Who won the civil war?'),
    ('Bob', 'How do we measure the distance between stars?'),
    ('Joe', 'How do we measure the distance between stars?'),
    ('Sally', 'How do we measure the distance between stars?'),
    ('Bob', 'Who are the most well known rappers?'),
    ('Joe', 'Who are the most well known rappers?'),
    ('Sally', 'Who are the most well known rappers?'),
])
sess.import_rel('Questions', questions)

'?TopicPrompt(Q,P)'

,Q,P
0,How do we measure the distance between stars?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: How do we measure the distance between stars?"
1,Who are the most well known rappers?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: Who are the most well known rappers?"
2,Who won the civil war?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: Who won the civil war?"


Now lets combine all of this into a pipeline.


In [ ]:
%%spannerlog
model = 'gpt-3.5-turbo'

TopicPrompt(Q,P)<-\
    Questions(user,Q),\
    Prompts('topic_selection',Template),\
    format(Template,Q)->(P)
    
?TopicPrompt(Q,P)

TopicSelection(Q,T)<-\
    TopicPrompt(Q,P),\
    llm($model,P)->(T)

?TopicSelection(Q,T)
StylePrompt(Q,P,Topic,user)<-\
    Questions(user,Q),\
    TopicSelection(Q,Topic),\
    UserAnswerPreference(user,UserStyle),\
    TopicSpecificStyle(Topic,TopicStyle),\
    Prompts('custom_style_prompt',promptTemplate),\
    format(promptTemplate,TopicStyle,UserStyle,Q)->(P)


Style_Based_QA(Q,Topic,user,P,A)<-\
    StylePrompt(Q,P,Topic,user),\
    llm($model,P)->(A)



And as you can see, we have a topic and user dependant LLM pipeline

In [ ]:
%%spannerlog
?Style_Based_QA(Q,Topic,user,Prompt,Answer)

'?Style_Based_QA(Q,Topic,user,Prompt,Answer)'

,Q,Topic,user,Prompt,Answer
0,How do we measure the distance between stars?,science,Bob,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: please answer in prose. Make sure you add references in the end like a bibliography. user: How do we measure the distance between stars?,"The distance between stars is measured in astronomical units (AU), parsecs (pc), or light-years (ly). 1. **Astronomical Unit (AU)**: An AU is the average distance between the Earth and the Sun, which is approximately 93 million miles or 150 million kilometers. It is more commonly used to measure distances within our solar system. 2. **Parsec (pc)**: A parsec is a unit of distance used in astronomy, defined as the distance at which an object would have a parallax angle of one arcsecond. One parsec is equal to about 3.26 light-years or 3.09 x 10^13 kilometers. 3. **Light-year (ly)**: A light-year is the distance that light travels in one year, which is approximately 9.46 trillion kilometers or about 5.88 trillion miles. To measure the distance to nearby stars, astronomers use a method called parallax. This involves observing a star from two different points in Earth's orbit around the Sun, which causes the star to appear to shift slightly against the background of more distant stars. By measuring the angle of this shift (the parallax angle), astronomers can calculate the distance to the star using trigonometry. For more distant stars, astronomers use other methods such as spectroscopic parallax, standard candles (like Cepheid variables), and the cosmic distance ladder which combines various distance measurement techniques to determine distances to faraway objects in the universe. References: 1. ""Astronomical unit"" - NASA, https://solarsystem.nasa.gov/solar-system/our-solar-system/overview/ 2. ""Parsec"" - European Space Agency, https://www.spacetelescope.org/about/general/parallax/ 3. ""Light-year"" - Space.com, https://www.space.com/15830-light-years.html 4. ""Parallax"" - European Southern Observatory, https://www.eso.org/public/usa/news/eso1718/"
1,How do we measure the distance between stars?,science,Joe,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: Prose is hard for me to read quickly. Format the answer in bullet points. user: How do we measure the distance between stars?,"- To measure the distance between stars, astronomers use a method called parallax, which is based on trigonometry. - Parallax involves observing a star from two different points in Earth's orbit and measuring the angle, known as the parallax angle, that the star appears to move. - The distance to the star can be calculated using the formula: distance = 1 / parallax angle. - The unit typically used for measuring stellar distances is the parsec (pc), where 1 parsec is equivalent to about 3.26 light years. - Other methods like photometric distance and spectroscopic parallax are also used to measure distances to stars that are much farther away."
2,How do we measure the distance between stars?,science,Sally,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: Try to avoid complicated jargon and use simple language. user: How do we measure the distance between stars?,"One common way to measure the distance between stars is through parallax. Parallax is the apparent shift in position of an object when viewed from two different points. The parallax angle, denoted by p, is the angle formed between a line from the observer to the star at two different times when Earth is on opposite sides of its orbit. The distance to the star, denoted by d, can be calculate